#### I imported necessary libraries**

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visulization
import seaborn as sns # data visulization
from copy import deepcopy # data copy
from sklearn.model_selection import train_test_split # data split
from sklearn.preprocessing import StandardScaler # data preprocessing
from sklearn.ensemble import RandomForestClassifier # machine learning algorithm
from sklearn.svm import SVC # machine learning algorithm
from sklearn.neighbors import KNeighborsClassifier # machine learning algorithm
from sklearn.linear_model import LogisticRegression # machine learning algorithm
from sklearn.metrics import confusion_matrix,roc_curve,roc_auc_score # measure error algorithm
from keras.models import Sequential # start ANN model Deep Learning tool
from keras.layers import Dense,Dropout # Deep Learning tool for prepare ANN model 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#### I loaded the dataset

In [ ]:
dataset=pd.read_csv("../input/heart-disease-uci/heart.csv")

### I got some information about the data

In [ ]:
dataset.info()

### I observed the data

In [ ]:
dataset.head()

#### I observed people that have heart dieases according to age

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(dataset.age[dataset.target == 1 ])
plt.title("Age Distribution of People That Being Heart Diases")
plt.ylabel("Number of People")
plt.xlabel("Age of People")
plt.show()

#### I observed people that have not heart dieases according to age

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(dataset.age[dataset.target == 0 ])
plt.title("Age Distribution of People That Not Being Heart Diases")
plt.ylabel("Number of People")
plt.xlabel("Age of People")
plt.show()

#### I divided it into labels and theirs features

In [ ]:
x=dataset.iloc[:,:-1].values
y=dataset.iloc[:,-1:].values
print("x shape :",x.shape)
print("y shape :",y.shape)

#### I divided the dataset as train and test dataset

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=0)
print("x_train shape :",x_train.shape)
print("x_test shape :",x_test.shape)
print("y_train shape :",y_train.shape)
print("y_test shape :",y_test.shape)

### I compressed the values -1 between 1

In [ ]:
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

# ARTIFICIAL NEURAL NETWORK

#### I creat my ANN model

In [ ]:
model = Sequential()

model.add(Dense(206,activation="relu",input_dim=13))
model.add(Dropout(0.4))
model.add(Dense(103,activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(1,activation="sigmoid"))

#### I compiled my model because I want to see validation and loss of train and test dataset

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

#### I started to work my model

In [ ]:
hist=model.fit(x_train,y_train,batch_size=250,epochs=100,validation_data=(x_test,y_test))

#### I equalized the values I obtained 0 and 1 for using heatmap visulization and I used deepcopy predicted values for use roc_curve and roc_auc_score algorithms 

In [ ]:
prediction=model.predict(x_test)
predict=deepcopy(prediction)
for i in range(0,len(prediction)):
    if prediction[i] <0.5:
        prediction[i] = 0
    else:
        prediction[i] = 1

#                         HEAT MAP

In [ ]:
cfm=confusion_matrix(y_test,prediction)
f,ax=plt.subplots(figsize=(10,10))
sns.heatmap(cfm,annot=True,cmap="coolwarm",linewidths=1,linecolor="black",fmt=".1f",ax=ax)
plt.title("Error Number With Heat Map")
plt.xlabel("Real")
plt.ylabel("Prediction")
plt.show()

# ROC CURVE

In [ ]:
fpr,tpr,threshold=roc_curve(y_test,predict)
print("fpr shape :",fpr.shape)
print("tpr shape :",tpr.shape)
print("threshold shape :",threshold.shape)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(tpr,color="green",label="TPR")
plt.plot(fpr,color="red",label="FPR")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.grid()
plt.title("Roc Curve Validation")
plt.show()

# ROC AUC SCORE

In [ ]:
score=roc_auc_score(y_test,predict)
print("Roc Auc Score :",score)

# RANDOM FOREST CLASSIFIER

#### I changed the number several numbers of estimators by for loop for which number to decide better

In [ ]:
train_score=[]
test_score=[]
for i in range(1,11):
 rfc=RandomForestClassifier(n_estimators=i,random_state=0)
 rfc.fit(x_train,y_train)
 train_score.append(rfc.score(x_train,y_train))
 test_score.append(rfc.score(x_test,y_test))

#### I observed and decided which number better

In [ ]:
plt.plot(train_score,color="green",label="Train Score")
plt.plot(test_score,color="red",label="Test Score")
plt.legend()
plt.xlabel("Number of Tree")
plt.ylabel("Score Validations")
plt.title("Choose Tree Number")
plt.show()

In [ ]:
rfc1=RandomForestClassifier(n_estimators=3,random_state=0)
rfc1.fit(x_train,y_train)
prediction=rfc1.predict(x_test)
print("Train Accuracy :",rfc1.score(x_train,y_train))
print("Test Accuracy :",rfc1.score(x_test,y_test))

# HEAT MAP

In [ ]:
f,ax=plt.subplots(figsize=(10,10))
cfm=confusion_matrix(y_test,prediction)
sns.heatmap(cfm,annot=True,cmap="coolwarm",linewidths=1,linecolor="black",fmt=".1f",ax=ax)
plt.title("Error Number With Heat Map")
plt.xlabel("Real")
plt.ylabel("Prediction")
plt.show()

# SUPPORT VECTOR CLASSIFIER

In [ ]:
svc=SVC(random_state=42)
svc.fit(x_train,y_train)
print("Train Accuracy :",svc.score(x_train,y_train))
print("Test Accuracy :",svc.score(x_test,y_test))

In [ ]:
prediction=svc.predict(x_test)

# HEAT MAP

In [ ]:
f,ax=plt.subplots(figsize=(10,10))
cfm=confusion_matrix(y_test,prediction)
sns.heatmap(cfm,annot=True,cmap="coolwarm",linewidths=1,linecolor="black",fmt=".1f",ax=ax)
plt.title("Error Number With Heat Map")
plt.xlabel("Real")
plt.ylabel("Prediction")
plt.show()

# K-NEAREST NEİGHBOURS

In [ ]:
train_score=[]
test_score=[]
for i in range(1,11):
    knn=KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train,y_train)
    train_score.append(knn.score(x_train,y_train))
    test_score.append(knn.score(x_test,y_test))

In [ ]:
plt.plot(train_score,color="green",label="Train Score")
plt.plot(test_score,color="red",label="Test Score")
plt.legend()
plt.xlabel("Number of Neighbours")
plt.ylabel("Score Validations")
plt.title("Choose Tree Number")
plt.show()

In [ ]:
knn1=KNeighborsClassifier(n_neighbors=6)
knn1.fit(x_train,y_train)
print("Train Accuracy :",knn1.score(x_train,y_train))
print("Test Accuracy :",knn1.score(x_test,y_test))

In [ ]:
prediction=knn1.predict(x_test)

# HEAT MAP

In [ ]:
f,ax=plt.subplots(figsize=(10,10))
cfm=confusion_matrix(y_test,prediction)
sns.heatmap(cfm,annot=True,cmap="coolwarm",linewidths=1,linecolor="black",fmt=".1f",ax=ax)
plt.title("Error Number With Heat Map")
plt.xlabel("Real")
plt.ylabel("Prediction")
plt.show()

# LOGISTIC REGRESSION

In [ ]:
lr=LogisticRegression()
lr.fit(x_train,y_train)
print("Train Accuracy :",lr.score(x_train,y_train))
print("Test Accuracy :",lr.score(x_test,y_test))

In [ ]:
prediction=lr.predict(x_test)

# HEAT MAP 

In [ ]:
f,ax=plt.subplots(figsize=(10,10))
cfm=confusion_matrix(y_test,prediction)
sns.heatmap(cfm,annot=True,cmap="coolwarm",linewidths=1,linecolor="black",fmt=".1f",ax=ax)
plt.title("Error Number With Heat Map")
plt.xlabel("Real")
plt.ylabel("Prediction")
plt.show()